<a href="https://colab.research.google.com/github/machine-perception-robotics-group/MPRGDeepLearningLectureNotebook/blob/master/13_rnn/06_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformer

2017年に発表されたTransformerは，CNNやRNNなどを用いずAttention機構のみを用いたモデルです．翻訳や文章生成などのタスクでRNNとSeq2seqモデルが主流でしたが，これらのモデルは逐次的に単語を処理するため学習時に並列計算できないという問題がありました．また，長文に対してAttentionが使われていましたが，このAttentionはほとんどRNNと一緒に使われていました．一方で，TransformerはAttention機構だけ使うことで，入出力の文章同士の広範囲な依存関係を捉える構造になっています．

モデルはSeq2seqと同様にエンコーダ・デコーダモデルです．エンコーダでは，Multi-Head AttentionとFeed Forwardのブロックを$N$回スタックする構造です．デコーダでは，それに加えMasked Multi-Head Attentionのブロックで構成されています．Masked Multi学習時，デコーダは自己回帰を使用せず，全ターゲットを同時に入力し，全ターゲットを同時に予測します．この時，予測すべきターゲットの情報が予測前のデコーダにリークしないようにMaskします．評価時は自己回帰でターゲットを生成します．

<img src="https://github.com/himidev/Lecture/blob/main/13_rnn/06_Transformer/trans.jpeg?raw=true" width = 50%>

# Attetionの種類

## Self-Attention
Self-AttentionはQuery，Key，Valueが全て同じ情報を使うAttentionです．Self-Attentionは言語の文法や照応関係を獲得するのにも使われています．このSelf-Attentionは汎用に使える仕組みで，エンコーダとデコーダどちらにも使われています．

<img src="https://github.com/himidev/Lecture/blob/main/13_rnn/06_Transformer/self-attention.jpeg?raw=true" width = 50%>



## Source-Target-Attention
Source-Target-AttentionはQueryとMemory(Key, Value)が異なる情報を使うAttenitonです．Source-Target-Attentionは基本的にデコーダで使われます．例えば，図のようにエンコーダに「お腹/が/減った」を入力した場合に，デコーダが「ラーメン/食べ/よう」を出力する時を考えましょう．この時，最初にデコーダは\<BOS>を入力した時に，エンコーダの入力に着目しながらラーメンを出力します．この例では「減った」に着目しています．次にラーメンを入力し，「減った」に着目しながら「食べ」を出力します．これを繰り返します．つまり，デコーダはある時刻$t$のターゲットを受け取って，エンコーダの入力に着目しながら$t+1$時刻のターゲットを予測します．

<img src="https://github.com/himidev/Lecture/blob/main/13_rnn/06_Transformer/source-target-attention.jpeg?raw=true" width = 50%>

# Scaled Dot-Product Attention
こちらは，所謂**Self-Attentionの中身**です．こちらはTransformerの鍵になっています．
数式は以下の通りです．
\begin{equation}
{\rm Attention}(Q, K, V)={\rm softmax} ( \frac{QK^{T}}{\sqrt{d_k} } ) V
\end{equation}
ここで，$Q, K, V$はそれぞれQuery，Key，Valueです．また$d_k$はQueryの次元数を表します．この平方根$d_k$は，見てわかるように$Q, K$の特徴量をスケールする役割を持ちます．これは層数，すなわちスタックするブロック数(前述のN)が大きくなると，内積が大きくなり，softmax関数の勾配を計算すると非常に小さい値しか返さないためです．

図のように，QueryとKeyが行列乗算で計算された後，dの平方根でスケーリングした後，後述するMaskをかけます．この時，Maskには負の無限大がかけられます．これにより，paddingした領域に対しsoftmax後の値を0に近い出力にすることができます．つまり，padding領域のAttention weightを計算しないようにします．最後にValueとの行列乗算をします．

<img src="https://github.com/himidev/Lecture/blob/main/13_rnn/06_Transformer/scale.jpeg?raw=true" width = 30%>

# Multi-Head Attention
Multi-Head Attentionは1つのQuery，Key，Valueを持たせるのではなく，小さいQuery，Key，Valueに分割して，分割した特徴表現を計算します．構造はシンプルで，Linear層とScaled Dot-Product Attentionを分割した構造を持ちます．最終的に，分割した出力を1つにまとめてLinear層に渡します．このようにわざわざ分割する理由ですが，モデルが異なる特徴表現の異なる情報についてAttention weightを計算できるためです．


<img src="https://github.com/himidev/Lecture/blob/main/13_rnn/06_Transformer/multi_head.jpeg?raw=true" width = 30%>

# Mask
冒頭でも述べたように，デコーダは未来の情報を伝播しないようにMaskをかけます．例をあげます．下図のように，Maskは同じ情報から作成されます．黒丸はマスクされた領域を表します．Maskは例えば，「好き」という情報を入力した場合に，残りの「な/動物/は」を参照できません．これは推論時未来の情報が与えられないためです．そのため，Queryでは，入力の時刻より先のMemoryの情報に対してMaskをすることで，未来の情報を伝播させないようにします．このマスクはデコーダのMasked Multi-Head Attentionで使われます．

<img src="https://github.com/himidev/Lecture/blob/main/13_rnn/06_Transformer/dec_pad.jpeg?raw=true" width = 50%>


次に，エンコーダ・デコーダに入力されるソースとターゲットの長さはバッチによって異なります．例えば，下図のように「おはよう」は１系列，「インコ/が/好き」は3系列，そして「お腹/減った」は2系列となっています．学習・推論時を一番長い3系列に合わせたい時，残りの1，2の系列をpaddingする必要があります．しかし，Attention weightを計算するときにpadding領域も計算されてしまうため，その領域がノイズとなり正確なAttentionを計算するのに邪魔になります．そのため，Attention weightを計算する際は，padding領域に対してMaskを適用します．基本的にこちらのマスクはMulti-Head Attention内で用います．


<img src="https://github.com/himidev/Lecture/blob/main/13_rnn/06_Transformer/enc_pad.jpeg?raw=true" width = 50%>

# ここからTransformerの実装に移ります

###データローダの作成
まず，データローダを用意します．データは0から9までの数字と加算記号，開始，終了のフラグです．また，３桁の数字の足し算を行うため，各桁の値を１つずつランダムに生成して連結しています．


In [ ]:
import sys
import numpy as np
import math 
import copy
from time import time
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

word2id = {str(i): i for i in range(10)}
word2id.update({"<pad>": 10, "+": 11, "<eos>": 12})
id2word = {v: k for k, v in word2id.items()}

class CalcDataset(torch.utils.data.Dataset):

    def transform(self, string, seq_len=7):
        tmp = []
        for i, c in enumerate(string):
            try:
                tmp.append(word2id[c])
            except:
                tmp += [word2id["<pad>"]] * (seq_len - i)
                break
        return tmp

    def __init__(self, data_num, train=True):
        super().__init__()
        self.data_num = data_num
        self.train = train
        self.data = []
        self.label = []

        for _ in range(data_num):
            x = int("".join([random.choice(list("0123456789")) for _ in range(random.randint(1, 3))] ))
            y = int("".join([random.choice(list("0123456789")) for _ in range(random.randint(1, 3))] ))
            left = ("{:*<7s}".format(str(x) + "+" + str(y))).replace("*", "<pad>")
            self.data.append(self.transform(left))

            z = x + y
            right = ("{:*<6s}".format(str(z))).replace("*", "<pad>")
            right = self.transform(right, seq_len=5)
            right = [12] + right
            right[right.index(10)] = 12
            self.label.append(right)
        


        self.data = np.asarray(self.data)
        self.label = np.asarray(self.label)

    def __getitem__(self, item):
        d = self.data[item]
        l = self.label[item]
        return d, l

    def __len__(self):
        return self.data.shape[0]

batch_size = 100
epoch_num = 200
train_data = CalcDataset(data_num = 20000)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

### Maskを作成

In [ ]:
def enc_mask(batch_size, src, size):
    mask = src == word2id["<pad>"]
    mask = mask.float().masked_fill(mask == 1, float(0.0)).masked_fill(mask == 0, float(1.0))
    return mask.view(mask.size(0), 1, mask.size(1))

def dec_mask(batch_size, size):
    mask = torch.triu(torch.ones(size, size), 1)
    mask = mask.float().masked_fill(mask == 0, float(1.0)).masked_fill(mask == 1, float(0.0))
    mask = mask.view(1, *mask.shape)
    mask = mask.expand(batch_size, *mask.shape[1:])
    return mask

def create_masks(batch_size, src, trg):
    src_mask = enc_mask(batch_size, src, src.size(1))

    if trg is not None:
        size = trg.size(1)
        np_mask = dec_mask(batch_size, size)
        trg_mask = np_mask
        
    else:
        trg_mask = None
    return src_mask, trg_mask

特徴ベクトルが$embedding \_ dim$，Multi-head Attentionのhead数が$heads$，layer数が$n \_ layers$です．

In [ ]:
embedding_dim = 512
n_layers = 6
heads = 8
vocab_size = len(word2id)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Multi-Head AttentionとSelf-Attention

In [ ]:
class Norm(nn.Module):
    def __init__(self, embedding_dim, eps = 1e-6):
        super().__init__()
    
        self.size = embedding_dim
        
        self.alpha = nn.Parameter(torch.ones(self.size))
        self.bias = nn.Parameter(torch.zeros(self.size))
        
        self.eps = eps
    
    def forward(self, x):
        norm = self.alpha * (x - x.mean(dim=-1, keepdim=True)) \
        / (x.std(dim=-1, keepdim=True) + self.eps) + self.bias
        return norm

def attention(q, k, v, d_k, mask=None, dec_mask=False):
    scores = torch.matmul(q, k.transpose(-2, -1)) /  math.sqrt(d_k)
    if mask is not None:
        if dec_mask:
            mask = mask.view(mask.size(0), 1, mask.size(1), mask.size(2))
        else:
            mask = mask.unsqueeze(1)
        scores = scores.masked_fill(mask == 0, -1e9)
    
    scores = F.softmax(scores, dim=-1)

    output = torch.matmul(scores, v)
    return output

class MultiHeadAttention(nn.Module):
    def __init__(self, heads, embedding_dim):
        super().__init__()
        
        self.embedding_dim = embedding_dim
        self.d_k = embedding_dim // heads
        self.h = heads
        
        self.q_linear = nn.Linear(embedding_dim, embedding_dim)
        self.v_linear = nn.Linear(embedding_dim, embedding_dim)
        self.k_linear = nn.Linear(embedding_dim, embedding_dim)
        
        self.out = nn.Linear(embedding_dim, embedding_dim)
    
    def forward(self, q, k, v, mask=None, dec_mask=False):
        
        bs = q.size(0)
        k = self.k_linear(k).view(bs, -1, self.h, self.d_k)
        q = self.q_linear(q).view(bs, -1, self.h, self.d_k)
        v = self.v_linear(v).view(bs, -1, self.h, self.d_k)
        
        k = k.transpose(1,2)
        q = q.transpose(1,2)
        v = v.transpose(1,2)
        
        scores = attention(q, k, v, self.d_k, mask, dec_mask)

        concat = scores.transpose(1,2).contiguous().view(bs, -1, self.embedding_dim)
        output = self.out(concat)
    
        return output

class FeedForward(nn.Module):
    def __init__(self, embedding_dim, d_ff=2048):
        super().__init__() 
    
        self.linear_1 = nn.Linear(embedding_dim, d_ff)
        self.linear_2 = nn.Linear(d_ff, embedding_dim)
    
    def forward(self, x):
        x = F.relu(self.linear_1(x))
        x = self.linear_2(x)
        return x

### Encoder-DecoderのLinear処理

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, embedding_dim, heads):
        super().__init__()
        self.norm_1 = Norm(embedding_dim)
        self.norm_2 = Norm(embedding_dim)
        self.attn = MultiHeadAttention(heads, embedding_dim)
        self.ff = FeedForward(embedding_dim)

    def forward(self, x, mask):
        x2 = self.norm_1(x)
        x = x + self.attn(x2,x2,x2,mask, dec_mask=False)
        x2 = self.norm_2(x)
        x = x + self.ff(x2)
        return x
    
class DecoderLayer(nn.Module):
    def __init__(self, embedding_dim, heads):
        super().__init__()
        self.norm_1 = Norm(embedding_dim)
        self.norm_2 = Norm(embedding_dim)
        self.norm_3 = Norm(embedding_dim)
        
        self.attn_1 = MultiHeadAttention(heads, embedding_dim)
        self.attn_2 = MultiHeadAttention(heads, embedding_dim)
        self.ff = FeedForward(embedding_dim)

    def forward(self, x, e_outputs, src_mask, trg_mask):
        x2 = self.norm_1(x)
        x = x + self.attn_1(x2, x2, x2, trg_mask, dec_mask=True)
        x2 = self.norm_2(x)
        x = x + self.attn_2(x2, e_outputs, e_outputs, src_mask, dec_mask=False)
        x2 = self.norm_3(x)
        x = x + self.ff(x2)
        return x

### Encoder-DecoderのEmbedding処理

In [ ]:
class Embedder(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.embedding_dim = embedding_dim
        # self.embed = nn.Embedding(vocab_size, embedding_dim, padding_idx=word2id["<pad>"]) なぜかpadding_indが機能しない． Why?
        self.embed = nn.Embedding(vocab_size, embedding_dim)
    def forward(self, x):
        # 仕方なく，こちらで0埋めする
        pad_index = x != word2id["<pad>"]
        pad_index = pad_index.view(*pad_index.shape, 1)
        return self.embed(x)*pad_index + 0.0

class PositionalEncoder(nn.Module):
    def __init__(self, embedding_dim, max_seq_len = 200):
        super().__init__()
        self.embedding_dim = embedding_dim
        pe = torch.zeros(max_seq_len, embedding_dim)
        for pos in range(max_seq_len):
            for i in range(0, embedding_dim, 2):
                pe[pos, i] = \
                math.sin(pos / (10000 ** ((2 * i)/embedding_dim)))
                pe[pos, i + 1] = \
                math.cos(pos / (10000 ** ((2 * (i + 1))/embedding_dim)))
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
 
    
    def forward(self, x):
        x = x * math.sqrt(self.embedding_dim)
        seq_len = x.size(1)
        pe = Variable(self.pe[:,:seq_len], requires_grad=False)
        if x.is_cuda:
            pe.cuda()
        x = x + pe
        return x

### Transformerモデル

In [ ]:
def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

class Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, N, heads):
        super().__init__()
        self.N = N
        self.embed = Embedder(vocab_size, embedding_dim)
        self.pe = PositionalEncoder(embedding_dim)
        self.layers = get_clones(EncoderLayer(embedding_dim, heads), N)
        self.norm = Norm(embedding_dim)
    def forward(self, src, mask):
        x = self.embed(src)
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x, mask)
        return self.norm(x)
    
class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, N, heads):
        super().__init__()
        self.N = N
        self.embed = Embedder(vocab_size, embedding_dim)
        self.pe = PositionalEncoder(embedding_dim)
        self.layers = get_clones(DecoderLayer(embedding_dim, heads), N)
        self.norm = Norm(embedding_dim)
    def forward(self, trg, e_outputs, src_mask, trg_mask):
        x = self.embed(trg)
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x, e_outputs, src_mask, trg_mask)
        return self.norm(x)

class Transformer(nn.Module):
    def __init__(self, vocab_size, embedding_dim, N, heads):
        super().__init__()
        self.encoder = Encoder(vocab_size, embedding_dim, N, heads)
        self.decoder = Decoder(vocab_size, embedding_dim, N, heads)
        self.out = nn.Linear(embedding_dim, vocab_size)
    def forward(self, src, trg, src_mask, trg_mask):
        e_outputs = self.encoder(src, src_mask)
        d_output = self.decoder(trg, e_outputs, src_mask, trg_mask)
        output = self.out(d_output)
        return output

def get_model(embedding_dim, heads, n_layers, vocab_size):
    
    assert embedding_dim % heads == 0

    model = Transformer(vocab_size, embedding_dim, n_layers, heads)
       
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p) 
    
    return model

model = get_model(embedding_dim, heads, n_layers, vocab_size).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=word2id["<pad>"])
optimizer = optim.Adam(model.parameters(), lr=0.001)

### 学習

In [ ]:
# GPUの確認
use_cuda = torch.cuda.is_available()
print('Use CUDA:', use_cuda)

all_losses = []
start = time()
for epoch in range(1, epoch_num+1):
    epoch_loss = 0
    for src, trg in train_loader:
        model.zero_grad()

        if use_cuda:
            src = src.cuda()
            trg = trg.cuda()

        trg_input = trg[:, :-1]
        src_mask, trg_mask = create_masks(batch_size, src, trg_input)
        if use_cuda:
          src_mask = src_mask.cuda()
          trg_mask = trg_mask.cuda()
        import sys
        print(src.shape, src_mask.shape)
        sys.exit()
        preds = model(src, trg_input, src_mask, trg_mask)
        loss = criterion(preds.view(-1, preds.size(-1)), trg[:, 1:].contiguous().view(-1))

        loss.backward()
        epoch_loss += loss.item()

        optimizer.step()

    elapsed_time = time() - start
    all_losses.append(epoch_loss)
    if epoch % 10 == 0:
        print("epoch: {}, mean loss: {}, elapsed_time: {}".format(epoch, loss.item(), elapsed_time))
        
model_name = "seq2seq_calculator_v{}.pt".format(epoch)
torch.save({
    'model': model.state_dict(),
}, model_name)


### 評価
今回は貪欲法で実装．本来のTransformerはビームサーチで行っているらしいが，割愛．

In [ ]:
batch_size = 1
test_data = CalcDataset(data_num = 2000)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)
model = get_model(embedding_dim, heads, n_layers, vocab_size).to(device)
model_name = "seq2seq_calculator_v{}.pt".format(200)
checkpoint = torch.load(model_name)
model.load_state_dict(checkpoint["model"])

accuracy = 0
        
# 評価の実行   
with torch.no_grad():
    for src, trg in test_loader:
        if use_cuda:
            src = src.cuda()
            trg = trg.cuda()

        trg_input = trg[:, :-1].clone()
        src_mask, trg_mask = create_masks(batch_size, src, trg_input)
        if use_cuda:
          src_mask = src_mask.cuda()
          trg_mask = trg_mask.cuda()

        # encoder
        e_output = model.encoder(src, src_mask)

        # decoder
        right = []
        for s in range(7):
            outputs = trg_input[:, :s+1]
            trg_mask_ = trg_mask[:, :s+1, :s+1]
            out = model.out(model.decoder(outputs, e_output, src_mask, trg_mask_))
            out = F.softmax(out, dim=2)

            if s == 0:
              index = torch.argmax(out.cpu().detach()).item()
            else:
              index = torch.argmax(out, dim=2)[0, -1].cpu().detach().item()
            token = id2word[index]

            if token == "<eos>":
                break
            right.append(token)

            if use_cuda:
              trg_input[:, s+1] = torch.LongTensor([word2id[token]]).cuda()
            else:
              trg_input[:, s+1] = torch.LongTensor([word2id[token]])
        right = "".join(right)

        if "+" in right or "<pad>" in right:
          accuracy += 0
          continue

        x = list(src[0].to('cpu').detach().numpy() )
        try:
            padded_idx_x = x.index(word2id["<pad>"])
        except ValueError:
            padded_idx_x = len(x)
        left = "".join(map(lambda c: str(id2word[c]), x[:padded_idx_x]))
        flag = ["F", "T"][eval(left) == int(right)]
        print("{:>7s} = {:>4s} :{}".format(left, right, flag))
        if flag == "T":
            accuracy += 1

print("Accuracy: {:.2f}".format(accuracy / len(test_loader)))


# 課題
* 四則演算を変えてみよう